In [1]:
from numiphy.symlib.expressions import *
from numiphy.odesolvers import *
import matplotlib.pyplot as plt

def g(x0):
    return [x0, 0, 0, (2 - 2*Vfunc(x0, 0) - 0**2)**0.5]

t, x, y = variables('t, x, y')
eps = 0.35
a = -Rational(1, 3)

V = x**2/2 + y**2/2 + eps*(x*y**2 + a*x**3)
ch = HamiltonianSystem(V, x, y)

Vfunc = V.lambdify([x, y])
orb1 = ch.new_varorbit(g(0.5), lowlevel=True)
orb2 = ch.new_varorbit(g(-0.05))

orb1.get(10000000, 0.01, err=1e-10, max_frames=2, split=10000)
orb2.get(10000000, 0.01, err=1e-10, max_frames=2, split=10000)

t1, ind1 = orb1.lyapunov
t2, ind2 = orb2.lyapunov

fig, ax = plt.subplots()
ax.plot(t1, ind1, label='Lissajous')
ax.plot(t2, ind2, label='Chaotic')
ax.grid(True)
ax.legend()

/tmp/tmppxgyxqzm/tmp_cpp_module0.cpp
Compiling...
Done


cc1plus: fatal error: /home/senpy/myenv/lib/python3.12/site-packages/numiphy/odesolvers/odepack/arrays.cpp: No such file or directory
compilation terminated.
/tmp/tmppxgyxqzm/tmp_cpp_module0.cpp:1:10: fatal error: /home/senpy/myenv/lib/python3.12/site-packages/numiphy/odesolvers/odepack/pyode.hpp: No such file or directory
    1 | #include "/home/senpy/myenv/lib/python3.12/site-packages/numiphy/odesolvers/odepack/pyode.hpp"
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
compilation terminated.


ImportError: /tmp/tmppxgyxqzm/ode_module0.cpython-312-x86_64-linux-gnu.so: cannot open shared object file: No such file or directory

In [2]:
os.path.exists("/home/senpy/myenv/lib/python3.12/site-packages/numiphy/odesolvers/odepack/arrays.cpp")

False